In [1]:
import time
import numpy as np
import pandas as pd

from pprint import pprint
from datetime import datetime, date, timedelta

# Spotify Specific Libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import API key
from config import POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_ADDRESS, POSTGRES_PORT, POSTGRES_DBNAME, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI

In [2]:
print(f"client_id = {SPOTIFY_CLIENT_ID} || secret = {SPOTIFY_CLIENT_SECRET}")

client_id = 5c2b054acc2345bfa97f51e3d172bcc5 || secret = 877d419ddb7c4cb69da44d02fbd033dd


In [3]:
from sqlalchemy import create_engine

# A long string that contains the necessary Postgres login information
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}"

# Create the connection
cnx = create_engine(postgres_str)

# TEST ONLY

### Proof of Concept: Read Data From DB

In [4]:
try:
    myTestdf = pd.read_sql_query("SELECT * FROM active_user LIMIT 5", cnx)
    
except Error as e:
    print(f"<message>./n/n  Error is:  {e}")
finally:
    print("Done.")
# end try

Done.


In [5]:
myTestdf.head()

,id,first_name,last_name,username
0,6,Fraser,Korneev,fstappard5
1,7,Demott,Rapson,lhambling6
2,15,Sadella,Jaram,wheinerte
3,17,Hewitt,Trammel,droughsedgeg
4,19,Ted,Knowlys,ydudeniei


# TEST - Spotify API

### Proof of Concept: Pull Data From Each Spotify API We Will Be Using

In [6]:
# on Mac, use `export` instead of `set`
# set SPOTIPY_CLIENT_ID = SPOTIFY_CLIENT_ID
# set SPOTIPY_CLIENT_SECRET = SPOTIFY_CLIENT_SECRET

baseURL = "https://api.spotify.com"

cid = SPOTIFY_CLIENT_ID
secret = SPOTIFY_CLIENT_SECRET

# Next, create a Spotify object and call methods:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [11]:
results = sp.search(q='stick figure', limit=1)
print(results)
# for idx, track in enumerate(results['tracks']['items']):
#     print(idx, track['name'])

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=stick+figure&type=track&offset=0&limit=1', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5SXEylV07TC57eanSxxg4R'}, 'href': 'https://api.spotify.com/v1/artists/5SXEylV07TC57eanSxxg4R', 'id': '5SXEylV07TC57eanSxxg4R', 'name': 'Stick Figure', 'type': 'artist', 'uri': 'spotify:artist:5SXEylV07TC57eanSxxg4R'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/

In [8]:
results = sp.category_playlists(category_id="party", country="US", limit=20, offset=0)
# for idx, track in enumerate(results['tracks']['items']):
#     print(idx, track['name'])
print(results)

KeyError: 'tracks'

In [9]:
print(results)

{'playlists': {'href': 'https://api.spotify.com/v1/browse/categories/party/playlists?country=US&offset=0&limit=20', 'items': [{'collaborative': False, 'description': "Turn up with the biggest party tracks of the '90s and '00s.", 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX8ky12eWIvcW'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX8ky12eWIvcW', 'id': '37i9dQZF1DX8ky12eWIvcW', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f0000000270360cd384e89a266c8480a1', 'width': None}], 'name': 'Throwback Jams', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTU4OTgwNzkxNCwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX8ky12eWIvcW/t

In [ ]:
# audio_analysis(track_id)
# Get audio analysis for a track based upon its Spotify ID Parameters:

# track_id - a track URI, URL or ID
# audio_features(tracks=[])
# Get audio features for one or multiple tracks based upon their Spotify IDs Parameters:

# tracks - a list of track URIs, URLs or IDs, maximum: 100 ids

In [13]:
trackID = 'spotify:track:3W4x7fZhMHw2pE6wyBzzF8'

trackResults = sp.audio_analysis(trackID)
print(trackResults)

{'meta': {'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': '1579338374', 'analysis_time': 826.93036, 'input_process': 'libvorbisfile L+R 44100->22050'}, 'track': {'num_samples': '9776522', 'duration': 443.37967, 'sample_md5': '', 'offset_seconds': 0, 'window_seconds': 0, 'analysis_sample_rate': 22050.0, 'analysis_channels': 1, 'end_of_fade_in': 0.0, 'start_of_fade_out': 425.11673, 'loudness': -7.327, 'tempo': 141.994, 'tempo_confidence': 0.9, 'time_signature': 4, 'time_signature_confidence': 1.0, 'key': 6, 'key_confidence': 0.636, 'mode': 1, 'mode_confidence': 0.561, 'codestring': 'eJxVnQlyJLuuQ7dSS9A87H9jjQOobHfE__d1ylk5UBIHEGSOVtcct5dP-bSy7qltjM845dN7X-O2ez5njc_Zdd02zvzUXtunjjnn2LN86m7ns8qord56P63q56fW2m-ruuS8jcM-at_r08vQhedstZbaP33M9TlntN3W_fSzm_44ZplltM9o53C419Apn7GL_tPq2aOU9pl16uQ-71y9rM-8J3-dOh6fdep6f-Uqu_X2ufWcU9bU4dLr6WZt6VL6V7nrM9YaVafr4afer-vx7tr3fs5d5XNPO7UUvdr1a8zZ9xmr8wfdd5659Zz1U0srfuiz1qxDx6shjnLu3Hd_9NpfAZS5j46RgF

In [17]:
trackResults = sp.audio_features(tracks = trackID)
print(trackResults)

[{'danceability': 0.857, 'energy': 0.713, 'key': 6, 'loudness': -7.327, 'mode': 1, 'speechiness': 0.0354, 'acousticness': 0.0665, 'instrumentalness': 0.0279, 'liveness': 0.0748, 'valence': 0.869, 'tempo': 141.994, 'type': 'audio_features', 'id': '3W4x7fZhMHw2pE6wyBzzF8', 'uri': 'spotify:track:3W4x7fZhMHw2pE6wyBzzF8', 'track_href': 'https://api.spotify.com/v1/tracks/3W4x7fZhMHw2pE6wyBzzF8', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3W4x7fZhMHw2pE6wyBzzF8', 'duration_ms': 443380, 'time_signature': 4}]


In [16]:
trackResults[0]["energy"]

0.713